<a href="https://colab.research.google.com/github/hanseul1215/ESAA_study/blob/master/0411_handson_unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 비지도 학습  
레이블이 없는 데이터의 학습
- 군집  ex) 데이터 분석, 고객 분류, 추천 시스템, 검색 엔진, 이미지 분할, 준지도 학습, 차원 축소
- 이상치 탐지 ex) 결함 제품 감지, 시계열 데이터 -> 새로운 트렌드
- 밀도 추정 ex) 이상치 탐지, 데이터 분석, 시각화

# 9.1 군집
- **고객 분류**  
고객을 구매 이력이나 웹사이트 내 행동 등을 기반으로 클러스터로 모음, 동일한 클러스터 내의 사용자가 좋아하는 컨텐츠를 추천하는 추천 시스템을 만듦
- **데이터 분석**  
클러스터 별로 따로 분석
- **차원 축소**
- **이상치 탐지**  
부정 거래 탐지
- **준지도 학습**  
레이블된 샘플이 적다면 군집 수행 이후 동일한 클러스터에 있는 모든 샘플에 레이블을 전파
- **검색 엔진**  
비슷한 이미지를 찾음
- **이미지 분할**  
색 기반으로 픽셀을 클러스터로 모으고, 각 픽셀의 색을 해당 클러스터의 평균 색으로 바꿈

## 9.1.1 K-평균
- 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플을 할당함

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

군집에서 각 샘플의 레이블은 알고리즘이 샘플에 할당한 클러스터의 인덱스

In [ ]:
y_pred

array([0, 4, 1, ..., 2, 1, 4], dtype=int32)

In [ ]:
# KMeans 클래스의 인스턴스는 labels_ 인스턴스 변수에 훈련된 샘플의 레이블을 가지고 있음
y_pred is kmeans.labels_

True

In [ ]:
kmeans.cluster_centers_

array([[-2.80037642,  1.30082566],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80389616,  1.80117999]])

In [ ]:
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

- 하드 군집 : 샘플을 하나의 클러스터에 할당하는 것
- 소프트 군집 : 클러스터마다 샘플에 점수를 부여하는 것

In [ ]:
# 샘플과 각 센트로이드 사이의 거리를 반환
kmeans.transform(X_new)

array([[2.88633901, 0.32995317, 2.9042344 , 1.49439034, 2.81093633],
       [5.84236351, 2.80290755, 5.84739223, 4.4759332 , 5.80730058],
       [1.71086031, 3.29399768, 0.29040966, 1.69136631, 1.21475352],
       [1.21567622, 3.21806371, 0.36159148, 1.54808703, 0.72581411]])

**K-평균 알고리즘**  
-> 센트로이드 랜덤 생성  
-> 샘플에 레이블 할당 후 센트로이드 업데이트  
-> 센트로이드에 변화가 없을 때까지 반복

In [ ]:
# 센트로이드 초기화
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42) 

**이너셔**  
각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리

In [ ]:
kmeans.fit(X) # n_init 중 이너셔가 가장 낮은 모델 반환
kmeans.inertia_

211.5985372581684

In [ ]:
# 이너셔의 음수값
kmeans.score(X)

-211.59853725816836

**k-평균 속도 개선과 미니배치**  
- 삼각 부등식을 사용하여 샘플과 센트로이드 사이 거리의 최솟값과 최댓값을 유지  
- 미니배치는 속도는 빠르지만 일반적으로 이너셔는 더 나빠 



In [ ]:
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5, random_state=42)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5, random_state=42)

최적의 클러스터 찾기
### 실루엣 점수
- 모든 샘플에 대한 실루엣 계수의 평균
- (b-a)/max(a, b)
- +1이면 클러스터 안에 잘 속해있다.
- -1이면 잘못된 클러스터에 할당


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

## 9.1.2 k-평균의 한계
- 최적의 솔루션 위해 알고리즘 여러 번 실행
- 클러스터 개수 지정
- 클러스터의 크기, 밀집도가 서로 다르거나, 원형이 아닐 경우 잘 작동X  

## 9.1.3 군집을 사용한 이미지 분할
-이미지 분할: 이미지를 세그먼트 여러 개로 분할
-시맨틱 분할: 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당
-색상 분할: 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당


## 9.1.4 군집을 사용한 전처리

In [ ]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
log_reg_score = log_reg.score(X_test, y_test)
log_reg_score

0.96

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("log_reg", LogisticRegression()),
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('kmeans', KMeans(n_clusters=50, random_state=42)),
                ('log_reg', LogisticRegression())])

In [ ]:
pipeline_score = pipeline.score(X_test, y_test)
pipeline_score

0.96

In [ ]:
# 최적의 k 찾기
from sklearn.model_selection import GridSearchCV
param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   0.5s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   1.3s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.5s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................kmeans__n_clusters=99; total time=   2.6s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans',
                                        KMeans(n_clusters=50, random_state=42)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [ ]:
grid_clf.best_params_

{'kmeans__n_clusters': 69}

In [ ]:
grid_clf.score(X_test, y_test)

0.9577777777777777

정확도 향상이 안되는데?

## 9.1.5 군집을 사용한 준지도 학습

레이블이 없는 샘플이 많고 레이블이 있는 샘플이 적을 때 사용

In [ ]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])
log_reg.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7822222222222223

In [ ]:
log_reg.score(X_test, y_test)

0.7822222222222223

In [ ]:
# 데이터를 50개의 클러스터로 모으고 각 클러스터에서 센트로이드에 가장 가까운 이미지를 찾아 -> 대표 이미지
# 대표 샘플에 레이블을 할당
k = 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

In [ ]:
# y_representative_digits = np.array([])

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

In [ ]:
# 레이블 전파 : 동일한 클러스터에 있는 모든 샘플로 전파
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [ ]:
# 센트로이드와 가까운 20%에만 레이블 전파
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.1111111111111111

## 9.1.6 DBSCAN
- 입실론 이웃 내에 적어도 min_samples개 샘플이 있다면 이를 핵심 샘플로 간주
- 핵심 샘플의 이웃에 있는 모든 샘플은 동일한 클러스터
- 핵심 샘플이 아니고, 이웃도 아닌 샘플은 이상치로 판단


In [2]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [3]:
dbscan.labels_

array([ 0,  0,  1,  2,  0,  2,  0,  0,  3,  5,  0,  0,  3,  4,  2,  3,  3,
        0,  0,  1,  3,  1,  4,  0,  3,  0,  3,  3,  1,  0,  0,  3,  3,  5,
        3,  0,  5,  6,  1,  4,  3,  4,  1,  8,  3,  5,  7,  3,  0,  0,  5,
        2,  3,  3,  4,  3,  0,  3,  2,  3,  2,  0,  3,  6,  2,  0,  4,  3,
        1, -1,  2,  3,  0,  4,  3,  2,  3,  3,  3,  3,  3,  0,  0,  2,  2,
       -1, -1,  2,  3,  0,  3,  0,  3,  0,  3,  2,  3,  1,  1,  0,  3,  4,
        0,  0,  0, -1,  0,  5,  4,  1,  3,  0,  3,  0,  0,  0,  0,  4,  0,
        0,  2,  5,  0,  1,  0,  2,  3,  3,  3, -1,  5,  0,  3,  3,  2,  0,
        3,  0,  4,  6,  2,  3,  7,  7,  2,  0,  3,  0, -1,  9, -1,  3,  3,
        4,  0,  0,  0,  3,  4,  0,  1,  3, -1,  0,  0,  0,  7,  3,  2,  2,
        4,  2,  1,  2,  2,  7,  7,  3,  1,  0,  0,  4,  2,  1,  0,  4, -1,
        1,  5,  0,  3,  0,  0,  4,  3,  0,  3,  4,  0,  2,  2,  3,  1,  6,
       -1,  0,  1,  0,  3,  3,  2,  1, -1,  0,  1,  0,  2,  7,  4,  3,  2,
        3,  3,  3, -1,  0

In [4]:
dbscan.core_sample_indices_# 핵심 샘플의 인덱스
dbscan.components_# 핵심 샘플

array([[ 0.92936779,  0.30786996],
       [-0.87084719,  0.57546791],
       [ 0.91435552, -0.52622845],
       ...,
       [-0.91041685,  0.23553184],
       [ 0.23168424, -0.07332775],
       [ 0.8927904 ,  0.39110064]])

dbscan은 새로운 샘플에 대해 클러스터를 예측X

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [7]:
import numpy as np
X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
knn.predict(X_new)

array([5, 4, 0, 3])

In [8]:
knn.predict_proba(X_new)

array([[0.  , 0.  , 0.  , 0.  , 0.16, 0.72, 0.  , 0.  , 0.12, 0.  ],
       [0.02, 0.  , 0.  , 0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.36, 0.  , 0.1 , 0.26, 0.  , 0.  , 0.28, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

## 9.1.7 다른 군집 알고리즘
- **병합 군집**  
클러스터 계층을 밑바닥부터 쌓아 구성하며 인접한 클러스터 쌍을 연결함   대규모 샘플에도 잘 적용됨
- **BIRCH**  
제한된 메모리를 사용해 대용량 데이터 셋을 다룸
- **평균-이동**  
각 샘플을 중심으로 하는 원을 그려 원 안의 모든 샘플의 평균을 구해 원의 중심을 평균점으로 이동시킴, 대규모 데이터 셋에는 적합X
- **유사도 전파**  
자신을 대표할 수 있는 비슷한 샘플에 투표
- **스펙트럼 군집**   
저차원 공간에서 또 다른 군집 알고리즘을 사용, 샘플 수가 많으면 잘 적용되지 않고, 클러스터 크기가 다르면 잘 동작X

# 9.2 가우시안 혼합 (GMM)
샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델
- 기댓값-최대화(EM) 알고리즘을 사용  
> 샘플을 클러스터에 할당한 뒤, 클러스터를 업데이트  
> 소프트 클러스터 할당을 사용 (각 클러스터에 속할 확률[책임] 예측)

In [9]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10, random_state=42)

In [10]:
gm.weights_

array([0.20270546, 0.20971816, 0.58757638])

In [11]:
gm.means_

array([[ 1.74171599, -0.05748783],
       [-0.72394873,  0.56875432],
       [ 0.51054353,  0.24334624]])

In [12]:
gm.covariances_

array([[[ 0.0504174 ,  0.06040572],
        [ 0.06040572,  0.08740914]],

       [[ 0.05829588,  0.0648149 ],
        [ 0.0648149 ,  0.08834599]],

       [[ 0.16787238, -0.10072011],
        [-0.10072011,  0.28784992]]])

In [13]:
gm.converged_

True

In [14]:
gm.n_iter_

16

In [15]:
gm.predict(X)

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 1,
       2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 0, 1, 0, 2, 1, 2, 1, 2, 0, 2, 1, 1,
       0, 1, 2, 0, 2, 2, 1, 2, 2, 0, 2, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2,
       2, 0, 1, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1,
       0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 0, 0, 1, 1, 1,
       0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 0, 2,
       2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 0, 2, 2, 2, 0, 1, 2, 2,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 0, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0,
       2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 0, 2, 2, 1,
       2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 1,
       2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2,

In [16]:
gm.predict_proba(X)

array([[9.77239712e-029, 5.43949495e-069, 1.00000000e+000],
       [2.52830691e-093, 1.69946103e-019, 1.00000000e+000],
       [3.94189972e-234, 9.97452264e-001, 2.54773602e-003],
       ...,
       [1.35932740e-204, 9.99535127e-001, 4.64873347e-004],
       [2.79791939e-056, 3.38086527e-042, 1.00000000e+000],
       [3.84897610e-034, 8.23333368e-062, 1.00000000e+000]])

In [17]:
gm.score_samples(X)

array([-1.47861054e+00, -1.85046032e+00, -9.90306763e-01, -1.83384312e+00,
       -2.11264062e+00, -1.68934008e+00, -1.41140680e+00, -1.79154669e+00,
       -2.14161036e+00, -1.10256594e+00, -1.84106039e+00, -1.55920017e+00,
       -8.71014612e-01, -1.71193787e+00, -1.92239960e+00, -1.14471004e+00,
       -3.30600145e-01, -1.52852209e+00, -1.72529117e+00, -6.80387789e-01,
       -2.17038389e+00, -1.65057672e-01, -1.81929653e+00, -1.40997846e+00,
       -2.20775171e+00, -8.84332184e-01, -8.08072784e-01, -7.01338088e-01,
        4.01008433e-02, -1.50572336e+00, -1.69760750e+00, -4.98653539e-02,
       -1.41450833e-01, -3.71356992e-01, -3.03632713e-01, -1.50870780e+00,
       -1.12366705e+00, -1.43901814e+00, -4.58734857e-01, -1.58702693e+00,
       -3.55660057e-01, -1.77333624e+00, -6.38106780e-01, -1.64380533e+00,
        8.18021405e-02, -2.31910173e-01, -1.75970821e+00, -7.25884108e-01,
       -1.38913243e+00, -1.68808721e+00, -2.29551375e-01, -1.61167862e+00,
       -2.24363949e+00, -

## 9.2.1 가우시안 혼합을 사용한 이상치 탐지
- 이상치 탐지: 보통과 많이 다른 샘플을 감지
- 특이치 탐지: 이상치로 오염되지 않은 데이터셋에서 훈련 

## 9.2.2 클러스터 개수 선택하기
- BIC or AIC 이용  
모두 학습할 파라미터가 많은 모델에게 벌칙을 가하고 데이터에 잘 학습하는 모델에게 보상을 더함
> **가능도 함수**
> 확률 : 세타를 알 때 출력 x가 얼마나 그럴듯한지 설명
> 가능도 : 출력 X를 알 때 세타가 얼마나 그럴듯한지 설명
> 세타의 최대가능도추정(MLE) / 사전 확률이 주어졌을때 최대사후확률(MAP)


In [19]:
print(gm.bic(X), gm.aic(X))

2748.9945615924257 2665.5627218497293


## 9.2.3 베이즈 가우시안 혼합 모델


In [20]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

array([0.15, 0.12, 0.13, 0.11, 0.16, 0.12, 0.1 , 0.1 , 0.  , 0.  ])

베타 분포는 고정 범위 안에 놓인 값을 가진 확률 변수를 모델링할 때 자주 사용  
베이즈 정리 중 분모 계산이 어려운 문제 해결 -> 변분 추론

## 9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘
- PCA  
- Fast-MCD  
알고리즘이 가우시안 분포의 파라미터(정상치)를 추정할 때 이상치로 의심되는 샘플을 무시
- 아이솔레이션 포레스트  
고차원 데이터셋에서 이상치 감지에 효과적
- LOF  
샘플 주위의 밀도와 이웃 주위의 밀도 비교
- one-class SVM  
특이치 탐지에 적합
